# Adaptive RAG

PDF 테이블 정보에 대한 Recursive Retrieval 전략
- 다수의 CSV 테이블 대상으로 검색 chunk와 답변 생성 chunk 분리해보기
- 테이블 검색용 Chunk 대상으로 User Query에 대해 Adaptive하게 retrieval 가져가보기


In [1]:
%pip install llama-index-embeddings-openai llama-index-llms-openai camelot-py llama-index pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.7/360.7 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 M

In [2]:
!apt-get install ghostscript
!pip install ghostscript

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 libgs9 libgs9-common libidn12 libijs-0.35
  libjbig2dec0 poppler-data
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre ghostscript-x poppler-utils
  fonts-japanese-mincho | fonts-ipafont-mincho fonts-japanese-gothic | fonts-ipafont-gothic
  fonts-arphic-ukai fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript libgs9 libgs9-common libidn12
  libijs-0.35 libjbig2dec0 poppler-data
0 upgraded, 10 newly installed, 0 to remove and 45 not upgraded.
Need to get 16.7 MB of archives.
After this operation, 63.0 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 fonts-droid-fallback all 1:6.0.1r16-1.1build1 [1,805 kB]
Ge

In [3]:
import camelot

from llama_index.core import VectorStoreIndex
from llama_index.core.query_engine import PandasQueryEngine
from llama_index.core.schema import IndexNode
from llama_index.llms.openai import OpenAI

from llama_index.readers.file import PyMuPDFReader
from typing import List

In [4]:
import os

os.environ["OPENAI_API_KEY"] = ''

In [5]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

# 추후 사용할 llm, 임베딩 모델 클래스 정의
Settings.llm = OpenAI(model="gpt-4o-mini", temperature=0)
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

In [6]:
# 파싱할 파일 경로 설정
file_path = "/content/billionaires_page.pdf"

In [7]:
# PDF파서 정의
reader = PyMuPDFReader()

In [8]:

# 업로드된 경로에서 로딩스테이지 진행한 후 다큐먼트 단위로 저장
docs = reader.load(file_path)

In [ ]:
# 도큐먼트 정보 확인
docs

[Document(id_='7f9efeb7-eadd-4aa2-9f7f-64b25b11f935', embedding=None, metadata={'total_pages': 32, 'file_path': '/content/billionaires_page.pdf', 'source': '1'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="The World's Billionaires\nList of the world's billionaires, ranked in order of net worth\nThe net worth of the world's billionaires increased from\nless than US$1 trillion in 2000 to over $7 trillion in 2015.\nPublication details\nPublisher\nWhale Media Investments\nForbes family\nPublication\nForbes\nFirst published\nMarch 1987[1]\nLatest publication\nApril 4, 2023\nCurrent list details (2023)[2]\nWealthiest\nBernard Arnault\nNet worth (1st)\n\xa0US$211\xa0billion\nNumber of\nbillionaires\n\xa02,640 (from 2668)\nTotal list net worth\nvalue\n\xa0US$12.2\xa0trillion (from US$12.7\ntrillion)\nNumber of women\n\xa0337\nNumber of men\n\xa02310\nNew members to the\nlist\n\xa0150\nForbes: The World's Billionaires website (https://www.forb\nes.com

In [9]:
from llama_index.core import Settings
#노드변환 및 파싱
doc_nodes = Settings.node_parser.get_nodes_from_documents(docs)

In [10]:
# 비교를 위한 Naive-RAG 구성
vector_index0 = VectorStoreIndex(doc_nodes)
vector_query_engine0 = vector_index0.as_query_engine()

In [11]:

response = vector_query_engine0.query(
    "How many billionaires were there in 2009?"
)

In [12]:
print(response.source_nodes[0].node.get_content())

No.
Name
Net worth
(USD)
Age
Nationality
Source(s) of wealth
1 
Carlos Slim
$74.0 billion 
71
 Mexico
América Móvil, Grupo Carso
2 
Bill Gates
$56.0 billion 
55
 United
States
Microsoft
3 
Warren Buffett
$50.0 billion 
80
 United
States
Berkshire Hathaway
4 
Bernard Arnault
$41.0 billion 
62
 France
LVMH Moët Hennessy • Louis
Vuitton
5 
Larry Ellison
$39.5 billion 
66
 United
States
Oracle Corporation
6 
Lakshmi Mittal
$31.1 billion 
60
 India
Arcelor Mittal
7 
Amancio Ortega
$31.0 billion 
74
 Spain
Inditex Group
8 
Eike Batista
$30.0 billion 
53
 Brazil
EBX Group
9 
Mukesh Ambani
$27.0 billion 
54
 India
Reliance Industries
10
Christy Walton &
family
$26.5 billion 
62
 United
States
Walmart
Slim narrowly eclipsed Gates to top the billionaire list for the first time. Slim saw his estimated worth surge
$18.5 billion to $53.5 billion as shares of America Movil rose 35 percent. Gates' estimated wealth rose $13
billion to $53 billion, placing him second. Warren Buffett was third with $47 

In [13]:
print(str(response))

The information provided does not specify the number of billionaires in 2009.


In [14]:
response = vector_query_engine0.query(
    "What's the net worth of the second richest billionaire in 2023?"
)
print(str(response))

The net worth of the second richest billionaire in 2023 is $201 billion.


In [15]:
print(response.source_nodes[0].node.get_content())

date roughly a month before publication. Privately held companies are priced by the prevailing price-to-
sales or price-to-earnings ratios. Known debt is subtracted from assets to get a final estimate of an
individual's estimated worth in United States dollars. Since stock prices fluctuate rapidly, an individual's true
wealth and ranking at the time of publication may vary from their situation when the list was compiled.[7]
When a living individual has dispersed his or her wealth to immediate family members it is included under a
single listing (as a single "family fortune") provided that individual (the grantor) is still living. However, if a
deceased billionaire's fortune has been dispersed, it will not appear as a single listing, and each recipient will
only appear if his or her own total net worth is over a $1 billion (his or her net worth will not be combined
with family members').[7] Royal families and dictators that have their wealth contingent on a position are
always excluded 

- 기본적인 PDF파싱모듈로는 테이블 등 Text-Only 가 아닌 문서에 대한 정보 해석력이 떨어지는 것을 확인
- Table정보를 따로 추출하여 답하는 방식은 어떨지?

In [16]:
# pdf의 테이블파싱하기
def get_tables(path: str, pages: List[int]):
    table_dfs = []
    for page in pages:
        table_list = camelot.read_pdf(path, pages=str(page))
        for table in table_list:
            table_df = table.df
            table_df = (
                table_df.rename(columns=table_df.iloc[0])
                .drop(table_df.index[0])
                .reset_index(drop=True)
            )
            table_dfs.append(table_df)
    return table_dfs

In [17]:
table_dfs = get_tables(file_path, pages=[3,4,24])

In [18]:
#파싱된 테이블 개수확인
len(table_dfs)

5

In [19]:
#파싱 결과 확인
table_dfs[0]

,No.,Name,Net worth\n(USD),Age,Nationality,Primary source(s) of wealth
0,1,Bernard Arnault &\nfamily,$211 billion,74,France,LVMH
1,2,Elon Musk,$180 billion,51,South Africa\n Canada\n United\nStates,"Tesla, SpaceX, Twitter (Currently\nX)"
2,3,Jeff Bezos,$114 billion,59,United\nStates,Amazon
3,4,Larry Ellison,$107 billion,78,United\nStates,Oracle Corporation
4,5,Warren Buffett,$106 billion,92,United\nStates,Berkshire Hathaway
5,6,Bill Gates,$104 billion,67,United\nStates,Microsoft
6,7,Michael Bloomberg,$94.5 billion,81,United\nStates,Bloomberg L.P.
7,8,Carlos Slim & family,$93 billion,83,Mexico,"Telmex, América Móvil, Grupo\nCarso"
8,9,Mukesh Ambani,$83.4 billion,65,India,Reliance Industries
9,10,Steve Ballmer,$80.7 billion,67,United\nStates,Microsoft


In [ ]:
#파싱 결과 확인
table_dfs[1]

,No.,Name,Net worth (USD),Age,Nationality,Primary source(s) of wealth
0,1,Elon Musk,$219 billion,50,South Africa\n Canada\n United States,"Tesla, SpaceX"
1,2,Jeff Bezos,$177 billion,58,United States,Amazon
2,3,Bernard Arnault & family,$158 billion,73,France,LVMH
3,4,Bill Gates,$129 billion,66,United States,Microsoft
4,5,Warren Buffett,$118 billion,91,United States,Berkshire Hathaway
5,6,Larry Page,$111 billion,49,United States,Google
6,7,Sergey Brin,$107 billion,48,United States,Google
7,8,Larry Ellison,$106 billion,77,United States,Oracle Corporation
8,9,Steve Ballmer,$91.4 billion,66,United States,Microsoft
9,10,Mukesh Ambani,$90.7 billion,64,India,Reliance Industries


In [ ]:
#파싱 결과 확인
table_dfs[-1]

,Year,Number of billionaires,Group's combined net worth
0,2023[2],"2,640",$12.2 trillion
1,2022[6],"2,668",$12.7 trillion
2,2021[11],"2,755",$13.1 trillion
3,2020,"2,095",$8.0 trillion
4,2019,"2,153",$8.7 trillion
5,2018,"2,208",$9.1 trillion
6,2017,"2,043",$7.7 trillion
7,2016,"1,810",$6.5 trillion
8,2015[18],"1,826",$7.1 trillion
9,2014[67],"1,645",$6.4 trillion


이제 테이블을 다 파싱해왔는데,
이것들을 기반으로 질문에 바로 답할수 있도록 만들려면 만들수도 있겠지만,
테이블이 지금과 다르게 수천 수만개일때, 모든 유저 쿼리에 대해 수만개의 테이블을 매번 조회하는 것은 실용성 없는 Naive한 접근방식(자원은 무한하지 않다).

그렇기 때문에,
1. 사용자의 질문과 관련된 테이블을 먼저 찾고
2. 찾은 테이블을 기준으로 사용자의 질문에 답할 수 있는 정보를 발췌하여 답해보자.

일단은 각 테이블별로 답해주는 담당 라마인덱스 쿼리엔진을 만들어주자.

(지난 주 커버한 쿼리엔진 라우터(AdaptiveRAG))

In [20]:
!pip install llama-index-experimental

In [21]:
from llama_index.experimental.query_engine import PandasQueryEngine

llm = OpenAI(model="gpt-4o-mini")

df_query_engines = [
    PandasQueryEngine(table_df, llm=llm) for table_df in table_dfs
]

In [22]:
df_query_engines

In [23]:
# 상응하는 테이블 직접 지정해서 답변 요구
response = df_query_engines[0].query(
    "What's the net worth of the second richest billionaire in 2023?"
)
print(str(response))

180.0


In [24]:
# 상응하는 테이블 직접 지정해서 답변 요구
response = df_query_engines[0].query(
    "Who is the richest guy in the world in 2023?"
)
print(str(response))

Bernard Arnault &
family


In [ ]:
table_dfs[0]

,No.,Name,Net worth\n(USD),Age,Nationality,Primary source(s) of wealth
0,1,Bernard Arnault &\nfamily,$211 billion,74,France,LVMH
1,2,Elon Musk,$180 billion,51,South Africa\n Canada\n United\nStates,"Tesla, SpaceX, Twitter (Currently\nX)"
2,3,Jeff Bezos,$114 billion,59,United\nStates,Amazon
3,4,Larry Ellison,$107 billion,78,United\nStates,Oracle Corporation
4,5,Warren Buffett,$106 billion,92,United\nStates,Berkshire Hathaway
5,6,Bill Gates,$104 billion,67,United\nStates,Microsoft
6,7,Michael Bloomberg,$94.5 billion,81,United\nStates,Bloomberg L.P.
7,8,Carlos Slim & family,$93 billion,83,Mexico,"Telmex, América Móvil, Grupo\nCarso"
8,9,Mukesh Ambani,$83.4 billion,65,India,Reliance Industries
9,10,Steve Ballmer,$80.7 billion,67,United\nStates,Microsoft


In [28]:
# 상응하는 테이블 지정해서 답변 요구
response = df_query_engines[2].query(
    "What's the net worth of the second richest billionaire in 2021?"
)
print(str(response))

151.0


In [26]:
# 상응하는 테이블 지정해서 답변 요구
response = df_query_engines[2].query(
    "where does this Jeff Bezos guy gets money from?"
)
print(str(response))

Amazon


In [ ]:
table_dfs[2]

,No.,Name,Net worth (USD),Age,Nationality,Source(s) of wealth
0,1,Jeff Bezos,$177 billion,57,United States,Amazon
1,2,Elon Musk,$151 billion,49,South Africa\n Canada\n United States,"Tesla, SpaceX"
2,3,Bernard Arnault & family,$150 billion,72,France,LVMH
3,4,Bill Gates,$124 billion,65,United States,Microsoft
4,5,Mark Zuckerberg,$97 billion,36,United States,Meta Platforms
5,6,Warren Buffett,$96 billion,90,United States,Berkshire Hathaway
6,7,Larry Ellison,$93 billion,76,United States,Oracle Corporation
7,8,Larry Page,$91.5 billion,48,United States,Google
8,9,Sergey Brin,$89 billion,47,United States,Google
9,10,Mukesh Ambani,$84.5 billion,63,India,Reliance Industries


In [29]:
response = df_query_engines[4].query(
    "How many billionaires were there in 2009?"
)
print(str(response))

793


In [30]:
response = df_query_engines[4].query(
    "How much is the 2009 billionaires' combined net worth?"
)
print(str(response))

$2.4 trillion


In [ ]:
table_dfs[4]

,Year,Number of billionaires,Group's combined net worth
0,2023[2],"2,640",$12.2 trillion
1,2022[6],"2,668",$12.7 trillion
2,2021[11],"2,755",$13.1 trillion
3,2020,"2,095",$8.0 trillion
4,2019,"2,153",$8.7 trillion
5,2018,"2,208",$9.1 trillion
6,2017,"2,043",$7.7 trillion
7,2016,"1,810",$6.5 trillion
8,2015[18],"1,826",$7.1 trillion
9,2014[67],"1,645",$6.4 trillion


질문별로 담당하는 쿼리엔진을 부여하는 것으로 heuristic하게 서칭 스페이스를 줄이고 시작할 수 있는 것 확인

In [31]:
# 쿼리엔진 요약문 생성
summaries = [
    (
        "This node provides information about the world's richest billionaires"
        " in 2023"
    ),
    (
        "This node provides information about the world's richest billionaires"
        " in 2022"
    ),
    (
        "This node provides information about the world's richest billionaires"
        " in 2021"
    ),
    (
        "This node provides information about the world's richest billionaires"
        " in 2020"
    ),
    (
        "This node provides information on the number of billionaires and"
        " their combined net worth from 2000 to 2023."
    ),
]

#생성된 요약문 별 노드단위 생성
df_nodes = [
    IndexNode(text=summary, index_id=f"pandas{idx}")
    for idx, summary in enumerate(summaries)
]

#요약노드 <-> 쿼리엔진 매핑
df_id_query_engine_mapping = {
    f"pandas{idx}": df_query_engine
    for idx, df_query_engine in enumerate(df_query_engines)
}

In [32]:
#생성된 노드 확인
df_nodes[0]

IndexNode(id_='962069f3-da0b-4e35-954e-5fef6a751517', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="This node provides information about the world's richest billionaires in 2023", mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n', index_id='pandas0', obj=None)

In [33]:
#상위레벨 벡터스토어인덱스 정의
vector_index = VectorStoreIndex(df_nodes)
vector_retriever = vector_index.as_retriever(similarity_top_k=1)

RecursiveRetriever 활용한 Adaptive Engine Selection
- Pydantic Selector과 다르게 LLM 기반의 Function Calling이 아닌 2 step retrieval.
- chunk retrieve 하듯이 쿼리엔진별 description 대상으로 1차 거리계산 retrieve(top_k=1), 이후 retrieve 된 description을 가진 쿼리 엔진을 이용하여 쿼리 답안 생성

In [34]:
from llama_index.core.retrievers import RecursiveRetriever

from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import get_response_synthesizer

recursive_retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever},
    query_engine_dict=df_id_query_engine_mapping,
    verbose=True,
)

response_synthesizer = get_response_synthesizer(response_mode="compact")

query_engine = RetrieverQueryEngine.from_args(
    recursive_retriever, response_synthesizer=response_synthesizer
)

In [35]:
response = query_engine.query(
    "What's the net worth of the second richest billionaire in 2023?"
)

Retrieving with query id None: What's the net worth of the second richest billionaire in 2023?
Retrieved node with id, entering: pandas0
Retrieving with query id pandas0: What's the net worth of the second richest billionaire in 2023?
Got response: 180.0


In [36]:
# 하위 리트리버의 response
response.source_nodes[0].node.get_content()

"Query: What's the net worth of the second richest billionaire in 2023?\nResponse: 180.0"

In [37]:
# 그걸 전달받은 상위 리트리버의 최종 답안
str(response)

'180.0'

In [38]:
response = query_engine.query("How many billionaires were there in 2009?")

Retrieving with query id None: How many billionaires were there in 2009?
Retrieved node with id, entering: pandas4
Retrieving with query id pandas4: How many billionaires were there in 2009?
Got response: 793


In [39]:
str(response)

'793'

In [44]:
# 파싱할 파일 경로 설정
file_path = "/content/son.pdf"

In [45]:
# PDF파서 정의
reader = PyMuPDFReader()
# 업로드된 경로에서 로딩스테이지 진행한 후 다큐먼트 단위로 저장
docs = reader.load(file_path)

In [46]:
doc_nodes = Settings.node_parser.get_nodes_from_documents(docs)

In [47]:
# 비교를 위한 Naive-RAG 구성
vector_index0 = VectorStoreIndex(doc_nodes)
vector_query_engine0 = vector_index0.as_query_engine()


In [48]:
response = vector_query_engine0.query(
    "손흥민이 소속된 구단이 어디야?"
)

In [49]:
print(str(response))

손흥민은 레버쿠젠과 토트넘 홋스퍼에서 활동한 것으로 보입니다.


In [50]:
print(response.source_nodes[0].node.get_content())

“손흥민, 2골 폭발…시즌 2-3호 기록” (http://www.tvreport.co.
kr/?c=news&m=newsview&idx=76966). TVREPORT.
26. 윤동빈 (2013년 2월 10일). “손흥민 8·9호 멀티골 폭발…獨언론 최고평점 부여” (http://news.chosu
n.com/site/data/html_dir/2013/02/10/2013021000293.html). 조선일보.
27. 김환 (2013년 4월 14일). “손흥민의 11호 골이 갖는 ‘세 가지 의미’” (https://news.naver.com/main/
read.nhn?mode=LSD&mid=sec&sid1=107&oid=241&aid=0002128435). 일간스포츠.
28. 정재훈 (2013년 6월 14일). “레버쿠젠 이적 확정, 손흥민 역할은 무엇?” (http://www.goal.com/kr/n
ews/1805/germany/2013/06/14/4046455/%EB%A0%88%EB%B2%84%EC%BF%A0%EC%
A0%A0-%EC%9D%B4%EC%A0%81-%ED%99%95%EC%A0%95-%EC%86%90%ED%9
D%A5%EB%AF%BC-%EC%97%AD%ED%95%A0%EC%9D%80-%EB%AC%B4%EC%9
7%87). 골닷컴. 2013년 6월 14일에 확인함.
29. 신민섭 (2013년 12월 1일). “해트트릭 이은 한경기 두 골, 손흥민 멀티 골 퍼레이드에 분데스리가 열
광” (https://web.archive.org/web/20131203013040/http://www.ilyo.co.kr/detail.php?number=6
7797&thread=04r02#). 일요신문. 2013년 12월 3일에 원본 문서 (http://www.ilyo.co.kr/detail.ph
p?number=67797&thread=04r02)에서 보존된 문서. 2013년 12월 1일에 확인함.
30. 김민지 (2013년 12월 1일). “손흥민 멀티골, 독일 현지 언

In [51]:
response = vector_query_engine0.query(
    "손흥민이 리그에서 23골을 넣었던 시즌이 언제야?"
)

In [52]:
print(str(response))

손흥민이 리그에서 23골을 넣었던 시즌은 2016-2017 시즌입니다.


In [53]:
print(response.source_nodes[0].node.get_content())

20. “손흥민“독일 함부르크 갑니다”” (https://news.naver.com/main/read.nhn?mode=LSD&mid=sec
&sid1=102&oid=087&aid=0000232331). 강원일보. 2009년 11월 11일.
21. “Glücklicher Cardoso lobt auch Son” (https://web.archive.org/web/20151122232543/http://sh.
sport-nord.de/news/index.php?news_id=13603&PHPSESSID=be51bc2569210df9e107afaf
d3487f89). 2015년 11월 22일에 원본 문서 (http://sh.sport-nord.de/news/index.php?news_id=1
3603&PHPSESSID=be51bc2569210df9e107afafd3487f89)에서 보존된 문서. 2015년 10월 30
일에 확인함.
22. 이용훈 (2010년 8월 5일). “함부르크 손흥민, 첼시 상대 역전 결승골” (http://www.goal.com/kr/new
s/1794/%ED%95%B4%EC%99%B8%ED%8C%8C/2010/08/05/2056387/%ED%95%A8%E
B%B6%80%EB%A5%B4%ED%81%AC-%EC%86%90%ED%9D%A5%EB%AF%BC-%E
C%B2%BC%EC%8B%9C-%EC%83%81%EB%8C%80-%EC%97%AD%EC%A0%84-%E
A%B2%B0%EC%8A%B9%EA%B3%A8). Goal.com. 2010년 8월 5일에 확인함.
23. Steve Han (2010년 10월 29일). “손흥민 "데뷔전, 아쉬움 남는 경기" ” (http://www.goal.com/kr/ne
ws/1794/%ED%95%B4%EC%99%B8%ED%8C%8C/2010/10/29/2188265/%EC%86%90%
ED%9D%A5%EB%AF%BC-%EB%8D%B0%EB%B7%94%EC%A0%84-%EC%95%84%E
C%89%AC%EC%9

In [54]:
print(response.source_nodes[1].node.get_content())

“손흥민, 2골 폭발…시즌 2-3호 기록” (http://www.tvreport.co.
kr/?c=news&m=newsview&idx=76966). TVREPORT.
26. 윤동빈 (2013년 2월 10일). “손흥민 8·9호 멀티골 폭발…獨언론 최고평점 부여” (http://news.chosu
n.com/site/data/html_dir/2013/02/10/2013021000293.html). 조선일보.
27. 김환 (2013년 4월 14일). “손흥민의 11호 골이 갖는 ‘세 가지 의미’” (https://news.naver.com/main/
read.nhn?mode=LSD&mid=sec&sid1=107&oid=241&aid=0002128435). 일간스포츠.
28. 정재훈 (2013년 6월 14일). “레버쿠젠 이적 확정, 손흥민 역할은 무엇?” (http://www.goal.com/kr/n
ews/1805/germany/2013/06/14/4046455/%EB%A0%88%EB%B2%84%EC%BF%A0%EC%
A0%A0-%EC%9D%B4%EC%A0%81-%ED%99%95%EC%A0%95-%EC%86%90%ED%9
D%A5%EB%AF%BC-%EC%97%AD%ED%95%A0%EC%9D%80-%EB%AC%B4%EC%9
7%87). 골닷컴. 2013년 6월 14일에 확인함.
29. 신민섭 (2013년 12월 1일). “해트트릭 이은 한경기 두 골, 손흥민 멀티 골 퍼레이드에 분데스리가 열
광” (https://web.archive.org/web/20131203013040/http://www.ilyo.co.kr/detail.php?number=6
7797&thread=04r02#). 일요신문. 2013년 12월 3일에 원본 문서 (http://www.ilyo.co.kr/detail.ph
p?number=67797&thread=04r02)에서 보존된 문서. 2013년 12월 1일에 확인함.
30. 김민지 (2013년 12월 1일). “손흥민 멀티골, 독일 현지 언

In [79]:
table_dfs = get_tables(file_path, pages=[14,15,19])

In [80]:
len(table_dfs)

3

In [81]:
#파싱 결과 확인
table_dfs[0]

,클럽,시즌,리그,,,,국내컵,,,리그컵,,,UEFA,,,합계,,
0,,,디비\n젼,경 기,골,도 움,경 기\n도 움\n골,,,경 기\n도 움\n골,,,도 움\n경기\n골,,,경 기,골,도 움
1,함부르\n크 SV\nII,2009-\n10,레기\n오날\n리가\n노드,6,1,0,—,,,—,,,—,,,6,1,0
2,함부르\n크 SV,2010-\n11,분데\n스리\n가,13,3,0,1,0,0,—,,,—,,,14,3,0
3,,2011-\n12,,27,5,1,3,0,0,—,,,—,,,30,5,1
4,,2012-\n13,,33,12,2,1,0,0,—,,,—,,,34,12,2
5,,합계,,73,20,3,5,0,0,—,,,—,,,78,20,3
6,바이어\n04 레\n버쿠젠,2013-\n14,분데\n스리\n가,31,10,4,4,2,1,—,,,8[a],0,2,43,12,7
7,,2014-\n15,,30,11,2,2,1,0,—,,,10[a],5,1,42,17,3
8,,2015-\n16,,1,0,0,0,0,0,—,,,1[a],0,0,2,0,0
9,,합계,,62,21,6,6,3,1,—,,,19,5,3,87,29,10


In [82]:
import pandas as pd
df = table_dfs[0].replace('\n', '', regex=True)

# Step 2: Forward fill missing '클럽' and '시즌' columns
df['클럽'] = df['클럽'].replace('', pd.NA).fillna(method='ffill')
df['시즌'] = df['시즌'].replace('', pd.NA).fillna(method='ffill')

# Step 3: Adjust columns based on the expected structure

# Since the structure is consistent, you can rename the columns for clarity
df.columns = [
    '클럽', '시즌', '리그', '리그_경기', '리그_골', '리그_도움',
    '국내컵_경기', '국내컵_골', '국내컵_도움', '리그컵_경기',
    '리그컵_골', '리그컵_도움', 'UEFA_경기', 'UEFA_골', 'UEFA_도움',
    '합계_경기', '합계_골', '합계_도움'
]

# Step 4: Replace dashes ('—') with NaN for better handling of missing data
df = df.replace('—', pd.NA)

# Step 5: Convert numeric columns to appropriate data types (if necessary)
# This step ensures that numeric data are treated as numbers, not strings
numeric_columns = [
    '리그_경기', '리그_골', '리그_도움',
    '국내컵_경기', '국내컵_골', '국내컵_도움',
    '리그컵_경기', '리그컵_골', '리그컵_도움',
    'UEFA_경기', 'UEFA_골', 'UEFA_도움',
    '합계_경기', '합계_골', '합계_도움'
]

df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')
df = df.drop(index=0)

# Reset the index after dropping the row
df = df.reset_index(drop=True)
table_dfs[0] = df

<ipython-input-82-d42b063bf33f>:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['클럽'] = df['클럽'].replace('', pd.NA).fillna(method='ffill')
<ipython-input-82-d42b063bf33f>:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['시즌'] = df['시즌'].replace('', pd.NA).fillna(method='ffill')


In [83]:
table_dfs[0]

,클럽,시즌,리그,리그_경기,리그_골,리그_도움,국내컵_경기,국내컵_골,국내컵_도움,리그컵_경기,리그컵_골,리그컵_도움,UEFA_경기,UEFA_골,UEFA_도움,합계_경기,합계_골,합계_도움
0,함부르크 SVII,2009-10,레기오날리가노드,6.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,1.0,0.0
1,함부르크 SV,2010-11,분데스리가,13.0,3.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,14.0,3.0,0.0
2,함부르크 SV,2011-12,,27.0,5.0,1.0,3.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,30.0,5.0,1.0
3,함부르크 SV,2012-13,,33.0,12.0,2.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,34.0,12.0,2.0
4,함부르크 SV,합계,,73.0,20.0,3.0,5.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,78.0,20.0,3.0
5,바이어04 레버쿠젠,2013-14,분데스리가,31.0,10.0,4.0,4.0,2.0,1.0,NaN,NaN,NaN,NaN,0.0,2.0,43.0,12.0,7.0
6,바이어04 레버쿠젠,2014-15,,30.0,11.0,2.0,2.0,1.0,0.0,NaN,NaN,NaN,NaN,5.0,1.0,42.0,17.0,3.0
7,바이어04 레버쿠젠,2015-16,,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,2.0,0.0,0.0
8,바이어04 레버쿠젠,합계,,62.0,21.0,6.0,6.0,3.0,1.0,NaN,NaN,NaN,19.0,5.0,3.0,87.0,29.0,10.0
9,토트넘홋스퍼,2015-16,프리미어리그,28.0,4.0,1.0,4.0,1.0,1.0,1.0,0.0,0.0,NaN,3.0,4.0,40.0,8.0,6.0


In [84]:
table_dfs[1]

,상대팀,골 수,날짜
0,사우샘프턴 FC,10,"2016, 05, 08\n2016, 12, 29\n2017, 12, 26\n2018..."
1,레스터 시티 FC,9,"2017, 05, 19\n2018, 12, 09\n2019, 02, 10\n2022..."
2,크리스탈 팰리스 FC,9,"2015, 09, 20\n2017, 11, 05\n2019, 04, 04\n2019..."
3,AFC 본머스,7,"2017, 04, 15\n2018, 03, 12\n2018, 12, 27\n2023..."
4,왓퍼드 FC,6,"2015, 12, 29\n2017, 04, 08\n2017, 12, 03\n2019..."
5,리버풀 FC,6,"2017, 10, 23\n2020, 12, 17\n2021, 12, 20\n2022..."
6,웨스트햄 유나이티드 FC,6,"2018, 01, 05\n2019, 11, 23\n2020, 10, 19\n2022..."
7,번리 FC,6,"2017, 04, 01\n2019, 12, 07\n2020, 10, 27\n2023..."
8,아스널 FC,6,"2020, 07, 13\n2020, 12, 07\n2021, 09, 27"


In [85]:
table_dfs[2]

,#,일시,장소,상대 국가,득점,결 과,매치 형식
0,1,2011년 1월\n18일,카타르 도하 타니 빈 자심 스타디\n움,인도,4-1,4-1,2011년 AFC 아시안컵
1,2,2013년 3월\n26일,대한민국 서울 서울월드컵경기장,카타르,2-1,2-1,2014년 FIFA 월드컵 아시아\n지역 4차 예선
2,3\n4,2013년 9월 6\n일,대한민국 인천 인천축구전용경기\n장,아이티,1-0\n4-1,4-1,친선경기
3,5,2013년 10월\n15일,대한민국 천안 천안종합운동장,말리,2-1,3-1,친선경기
4,6,2014년 3월 5\n일,그리스 아테네 카라이스카키스\n스타디움,그리스,2-0,2-0,친선경기
5,7,2014년 6월\n22일,브라질 포르투알레그리 이스타지\n우 베이라히우,알제리,1-3,2-4,2014년 FIFA 월드컵
6,8\n9,2015년 1월\n22일,오스트레일리아 멜버른 멜버른\n렉탱귤러 스타디움,우즈베키\n스탄,1-0\n2-0,2-0,2015년 AFC 아시안컵
7,10,2015년 1월\n31일,오스트레일리아 시드니 스타디움\n오스트레일리아,오스트레\n일리아,1-1,1-2,2015년 AFC 아시안컵
8,11,2015년 6월\n16일,태국 방콕 라차망칼라 스타디움,미얀마,2-0,2-0,2018년 FIFA 월드컵 아시아\n지역 2차 예선
9,12\n13\n14,2015년 9월 3\n일,대한민국 화성 화성종합경기타운,라오스,2-0\n5-0\n7-0,8-0,2018년 FIFA 월드컵 아시아\n지역 2차 예선


In [86]:
df_query_engines = [
    PandasQueryEngine(table_df, llm=llm) for table_df in table_dfs
]

In [87]:
# 상응하는 테이블 직접 지정해서 답변 요구
response = df_query_engines[0].query(
    "손흥민이 리그에서 23골을 넣었던 시즌이 언제야?"
)
print(str(response))

2021-22


In [133]:
# 상응하는 테이블 직접 지정해서 답변 요구
response = df_query_engines[1].query(
    "손흥민이 상대팀 리버풀 상대로 몇골 넣었지?"
)
print(str(response))

6


In [137]:
# 상응하는 테이블 직접 지정해서 답변 요구
response = df_query_engines[1].query(
    "손흥민이 어떤 팀 상대로 가장 많은 골을 넣었지?"
)
print(str(response))

사우샘프턴 FC


In [97]:
# 상응하는 테이블 직접 지정해서 답변 요구
response = df_query_engines[2].query(
    "2018 FIFA 월드컵에서 어떤 팀들 상대로 골 넣었었지?"
)
print(str(response))

15    멕시코
16     독일
Name: 상대 국가, dtype: object


In [99]:
table_dfs[0]

,클럽,시즌,리그,리그_경기,리그_골,리그_도움,국내컵_경기,국내컵_골,국내컵_도움,리그컵_경기,리그컵_골,리그컵_도움,UEFA_경기,UEFA_골,UEFA_도움,합계_경기,합계_골,합계_도움
0,함부르크 SVII,2009-10,레기오날리가노드,6.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,1.0,0.0
1,함부르크 SV,2010-11,분데스리가,13.0,3.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,14.0,3.0,0.0
2,함부르크 SV,2011-12,,27.0,5.0,1.0,3.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,30.0,5.0,1.0
3,함부르크 SV,2012-13,,33.0,12.0,2.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,34.0,12.0,2.0
4,함부르크 SV,합계,,73.0,20.0,3.0,5.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,78.0,20.0,3.0
5,바이어04 레버쿠젠,2013-14,분데스리가,31.0,10.0,4.0,4.0,2.0,1.0,NaN,NaN,NaN,NaN,0.0,2.0,43.0,12.0,7.0
6,바이어04 레버쿠젠,2014-15,,30.0,11.0,2.0,2.0,1.0,0.0,NaN,NaN,NaN,NaN,5.0,1.0,42.0,17.0,3.0
7,바이어04 레버쿠젠,2015-16,,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,2.0,0.0,0.0
8,바이어04 레버쿠젠,합계,,62.0,21.0,6.0,6.0,3.0,1.0,NaN,NaN,NaN,19.0,5.0,3.0,87.0,29.0,10.0
9,토트넘홋스퍼,2015-16,프리미어리그,28.0,4.0,1.0,4.0,1.0,1.0,1.0,0.0,0.0,NaN,3.0,4.0,40.0,8.0,6.0


In [124]:
# 쿼리엔진 요약문 생성
summaries = [
    (
        "This node provides information about 손흥민의 시즌별 통산 득점 기록"
    ),
    (
        "This node provides information about 손흥민의 상대팀별 기록."
    ),
    (
        "This node provides information about 손흥민의 국가대표팀 득점 기록"
    ),
]

#생성된 요약문 별 노드단위 생성
df_nodes = [
    IndexNode(text=summary, index_id=f"pandas{idx}")
    for idx, summary in enumerate(summaries)
]

#요약노드 <-> 쿼리엔진 매핑
df_id_query_engine_mapping = {
    f"pandas{idx}": df_query_engine
    for idx, df_query_engine in enumerate(df_query_engines)
}

In [125]:
#상위레벨 벡터스토어인덱스 정의
vector_index = VectorStoreIndex(df_nodes)
vector_retriever = vector_index.as_retriever(similarity_top_k=1)

In [126]:
recursive_retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever},
    query_engine_dict=df_id_query_engine_mapping,
    verbose=True,
)

response_synthesizer = get_response_synthesizer(response_mode="compact")

query_engine = RetrieverQueryEngine.from_args(
    recursive_retriever, response_synthesizer=response_synthesizer
)

In [132]:
response = query_engine.query(
    "손흥민이 상대팀 리버풀 상대로 몇골 넣었지?"
)
print(str(response))

Retrieving with query id None: 손흥민이 상대팀 리버풀 상대로 몇골 넣었지?
Retrieved node with id, entering: pandas1
Retrieving with query id pandas1: 손흥민이 상대팀 리버풀 상대로 몇골 넣었지?
Got response: 6
6


In [131]:
response = query_engine.query(
    "손흥민이 리그에서 23골을 넣었던 시즌이 언제야?"
)
print(str(response))

Retrieving with query id None: 손흥민이 리그에서 23골을 넣었던 시즌이 언제야?
Retrieved node with id, entering: pandas0
Retrieving with query id pandas0: 손흥민이 리그에서 23골을 넣었던 시즌이 언제야?
Got response: 2021-22
2021-22


In [136]:
response = query_engine.query(
    "손흥민이 어떤 팀 상대로 가장 많은 골을 넣었지?"
)
print(str(response))


Retrieving with query id None: 손흥민이 어떤 팀 상대로 가장 많은 골을 넣었지?
Retrieved node with id, entering: pandas1
Retrieving with query id pandas1: 손흥민이 어떤 팀 상대로 가장 많은 골을 넣었지?
Got response: 사우샘프턴 FC
사우샘프턴 FC


In [138]:

response = query_engine.query(
    "2018 FIFA 월드컵에서 어떤 팀들 상대로 골 넣었었지?"
)
print(str(response))


Retrieving with query id None: 2018 FIFA 월드컵에서 어떤 팀들 상대로 골 넣었었지?
Retrieved node with id, entering: pandas2
Retrieving with query id pandas2: 2018 FIFA 월드컵에서 어떤 팀들 상대로 골 넣었었지?
Got response: 15    멕시코
16     독일
Name: 상대 국가, dtype: object
멕시코, 독일
